In [1]:
# mlflow ui --backend-store-uri sqlite:///mlflow.db

In [2]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import root_mean_squared_error

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

import pickle

import mlflow

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-duration-prediction")

2024/06/17 19:21:56 INFO mlflow.tracking.fluent: Experiment with name 'nyc-duration-prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/experiment-tracking/mlruns/1', creation_time=1718644916557, experiment_id='1', last_update_time=1718644916557, lifecycle_stage='active', name='nyc-duration-prediction', tags={}>

In [4]:
def read_dataframe(url):
  df = pd.read_parquet(url)

  # compute duration in minutes
  df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
  df.duration = df.duration.apply(lambda x: x.total_seconds() / 60)

  # filter data based on trip duration
  df = df.loc[(df.duration >= 1) & (df.duration <= 60), :]

  return df

In [5]:
df_train = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

In [6]:
def feature_preparation(df, dv=None):
  # combine pickup and dropoff locations
  df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].astype(str)
  df['PU_DO'] = df.PULocationID + '_' + df.DOLocationID
  
  # dataframe to dictionaries
  data_dicts = df[['PU_DO', 'trip_distance']].to_dict(orient='records')

  if dv:
    X = dv.transform(data_dicts)
  else:
    dv = DictVectorizer()
    X = dv.fit_transform(data_dicts)
    
  y = df['duration'].values

  return X, y, dv

In [7]:
X_train, y_train, dv = feature_preparation(df=df_train)
X_val, y_val, _ = feature_preparation(df=df_val, dv=dv)

#### Manual Logging (Lasso)

In [8]:
# model params
alpha = .01

# model training
model = Lasso()
model.fit(X_train, y_train)

# model error tracking
preds = model.predict(X_val)
rmse = root_mean_squared_error(y_val, preds)

with mlflow.start_run():
  mlflow.set_tag('developer', 'Armand Winant')

  # data versioning
  mlflow.log_param('training-data-path', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
  mlflow.log_param('validation-data-path', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

  # model parameter tracking
  mlflow.log_param('alpha', alpha)
  mlflow.log_metric('rmse', rmse)

  mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

#### Hyperparameter Logging (XGBoost)

In [9]:
train = xgb.DMatrix(data=X_train, label=y_train)
valid = xgb.DMatrix(data=X_val, label=y_val)

In [10]:
def objective(params):
  booster = xgb.train(
    params = params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50)

  y_pred = booster.predict(valid)
  rmse = root_mean_squared_error(y_val, y_pred)

  with mlflow.start_run():
    mlflow.set_tag("model", "xgboost")
    mlflow.log_params(params)

    mlflow.log_metric("rmse", rmse)
  
  return { 'loss': rmse, 'status': STATUS_OK }

In [11]:
search_space = {
  'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
  'learning_rate': hp.loguniform('learning_rate', -3, 0),
  'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
  'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
  'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
  'objective': 'reg:linear',
  'seed': 42
}

best_result = fmin(
  fn=objective,
  space=search_space,
  algo=tpe.suggest,
  max_evals=50,
  trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:22:01] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.09480                          
[1]	validation-rmse:8.74886                           
[2]	validation-rmse:7.92465                           
[3]	validation-rmse:7.43235                           
[4]	validation-rmse:7.13944                           
[5]	validation-rmse:6.96444                           
[6]	validation-rmse:6.85563                           
[7]	validation-rmse:6.78569                           
[8]	validation-rmse:6.74116                           
[9]	validation-rmse:6.70985                           
[10]	validation-rmse:6.68719                          
[11]	validation-rmse:6.67182                          
[12]	validation-rmse:6.65821                          
[13]	validation-rmse:6.64799                          
[14]	validation-rmse:6.63995                          
[15]	validation-rmse:6.63285                          
[16]	validation-rmse:6.62641                          
[17]	validation-rmse:6.62130                          
[18]	valid

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [19:22:50] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.06736                                                    
[1]	validation-rmse:6.75821                                                    
[2]	validation-rmse:6.71508                                                    
[3]	validation-rmse:6.70666                                                    
[4]	validation-rmse:6.70073                                                    
[5]	validation-rmse:6.69015                                                    
[6]	validation-rmse:6.68748                                                    
[7]	validation-rmse:6.68602                                                    
[8]	validation-rmse:6.68721                                                    
[9]	validation-rmse:6.67937                                                    
[10]	validation-rmse:6.67436                                                   
[11]	validation-rmse:6.66988                                                   
[12]	validation-rmse:6.66958            

...

[0]	validation-rmse:9.54263                                                       
[1]	validation-rmse:8.07722                                                       
[2]	validation-rmse:7.31815                                                       
[3]	validation-rmse:6.93353                                                       
[4]	validation-rmse:6.73718                                                       
[5]	validation-rmse:6.62997                                                       
[6]	validation-rmse:6.56770                                                       
[7]	validation-rmse:6.52709                                                       
[8]	validation-rmse:6.50514                                                       
[9]	validation-rmse:6.48877                                                       
[10]	validation-rmse:6.47610                                                      
[11]	validation-rmse:6.46706                                                      
[12]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:27:13] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.44120                                                      
[1]	validation-rmse:10.76519                                                      
[2]	validation-rmse:10.17398                                                      
[3]	validation-rmse:9.65989                                                       
[4]	validation-rmse:9.21497                                                       
[5]	validation-rmse:8.83127                                                       
[6]	validation-rmse:8.49882                                                       
[7]	validation-rmse:8.21320                                                       
[8]	validation-rmse:7.97210                                                       
[9]	validation-rmse:7.76280                                                       
[10]	validation-rmse:7.58423                                                      
[11]	validation-rmse:7.43282                                                      
[12]

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:28:47] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.46664                                                      
[1]	validation-rmse:9.22719                                                       
[2]	validation-rmse:8.38002                                                       
[3]	validation-rmse:7.79452                                                       
[4]	validation-rmse:7.40682                                                       
[5]	validation-rmse:7.13732                                                       
[6]	validation-rmse:6.96869                                                       
[7]	validation-rmse:6.85234                                                       
[8]	validation-rmse:6.76876                                                       
[9]	validation-rmse:6.70791                                                       
[10]	validation-rmse:6.66554                                                      
[11]	validation-rmse:6.63203                                                      
[12]

#### MLFlow AutoLogging

In [12]:
mlflow.xgboost.autolog()

In [13]:
# get set of optimal params from mlflow ui
params = {
  'learning_rate': 0.19059089830787454,
  'max_depth': 39,
  'min_child_weight': 1.1436471134450543,
  'objective': 'reg:linear',
  'reg_alpha': 0.3184226350656614,
  'reg_lambda': 0.01269595936592078,
  'seed': 42
}

booster = xgb.train(
  params = params,
  dtrain = train,
  num_boost_round = 1000,
  evals=[(valid, "validation")],
  early_stopping_rounds=50
)

y_pred = booster.predict(valid)
rmse = root_mean_squared_error(y_val, y_pred)

with mlflow.start_run():
  mlflow.log_metric("rmse", rmse)

2024/06/17 20:29:41 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'cfb62812f52348858b0d65a905ea29dd', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:29:52] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.70401
[1]	validation-rmse:9.56757
[2]	validation-rmse:8.72686
[3]	validation-rmse:8.11361
[4]	validation-rmse:7.67338
[5]	validation-rmse:7.35519
[6]	validation-rmse:7.12959
[7]	validation-rmse:6.96680
[8]	validation-rmse:6.85015
[9]	validation-rmse:6.76309
[10]	validation-rmse:6.70056
[11]	validation-rmse:6.65235
[12]	validation-rmse:6.61576
[13]	validation-rmse:6.58891
[14]	validation-rmse:6.56640
[15]	validation-rmse:6.54774
[16]	validation-rmse:6.53385
[17]	validation-rmse:6.52169
[18]	validation-rmse:6.51257
[19]	validation-rmse:6.50470
[20]	validation-rmse:6.49959
[21]	validation-rmse:6.49550
[22]	validation-rmse:6.49190
[23]	validation-rmse:6.48876
[24]	validation-rmse:6.48704
[25]	validation-rmse:6.48499
[26]	validation-rmse:6.48387
[27]	validation-rmse:6.48226
[28]	validation-rmse:6.47952
[29]	validation-rmse:6.47729
[30]	validation-rmse:6.47489
[31]	validation-rmse:6.47331
[32]	validation-rmse:6.47229
[33]	validation-rmse:6.47021
[34]	validation-rmse:6.

2024/06/17 20:31:28 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/06/17 20:31:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:31:29] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


#### Model Logging

In [14]:
mlflow.xgboost.autolog(disable=True)

In [15]:
# get set of optimal params from mlflow ui
params = {
  'learning_rate': 0.19059089830787454,
  'max_depth': 39,
  'min_child_weight': 1.1436471134450543,
  'objective': 'reg:linear',
  'reg_alpha': 0.3184226350656614,
  'reg_lambda': 0.01269595936592078,
  'seed': 42
}

booster = xgb.train(
  params = params,
  dtrain = train,
  num_boost_round = 1000,
  evals=[(valid, "validation")],
  early_stopping_rounds=50
)

y_pred = booster.predict(valid)
rmse = root_mean_squared_error(y_val, y_pred)

# store the vectorizer object locally
with open("models/preprocessor.p", "wb") as f_out:
  pickle.dump(dv, f_out)

with mlflow.start_run():
  mlflow.log_params(params=params)
  mlflow.log_metric("rmse", rmse)

  # save the local vectorizer and model in mlflow
  mlflow.log_artifact("models/preprocessor.p", artifact_path="preprocessor")
  mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:31:40] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.70401
[1]	validation-rmse:9.56757
[2]	validation-rmse:8.72686
[3]	validation-rmse:8.11361
[4]	validation-rmse:7.67338
[5]	validation-rmse:7.35519
[6]	validation-rmse:7.12959
[7]	validation-rmse:6.96680
[8]	validation-rmse:6.85015
[9]	validation-rmse:6.76309
[10]	validation-rmse:6.70056
[11]	validation-rmse:6.65235
[12]	validation-rmse:6.61576
[13]	validation-rmse:6.58891
[14]	validation-rmse:6.56640
[15]	validation-rmse:6.54774
[16]	validation-rmse:6.53385
[17]	validation-rmse:6.52169
[18]	validation-rmse:6.51257
[19]	validation-rmse:6.50470
[20]	validation-rmse:6.49959
[21]	validation-rmse:6.49550
[22]	validation-rmse:6.49190
[23]	validation-rmse:6.48876
[24]	validation-rmse:6.48704
[25]	validation-rmse:6.48499
[26]	validation-rmse:6.48387
[27]	validation-rmse:6.48226
[28]	validation-rmse:6.47952
[29]	validation-rmse:6.47729
[30]	validation-rmse:6.47489
[31]	validation-rmse:6.47331
[32]	validation-rmse:6.47229
[33]	validation-rmse:6.47021
[34]	validation-rmse:6.

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:32:21] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


#### Loading model from MLFlow as Python object

In [17]:
logged_model = 'runs:/9dea084aee9c4779b14cc11ad63f9cc6/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# loaded_model.predict(pd.DataFrame(data))
preds = loaded_model.predict(X_val)

preds

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:33:02] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


array([14.245975,  6.961786, 15.038191, ..., 13.460661,  6.37222 ,
        8.019436], dtype=float32)

#### Loading model from MLFlow as XGB object

In [18]:
xgboost_model = mlflow.xgboost.load_model(model_uri=logged_model)

preds = xgboost_model.predict(valid)
preds

/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [20:33:12] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


array([14.245975,  6.961786, 15.038191, ..., 13.460661,  6.37222 ,
        8.019436], dtype=float32)